In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv("csvs\heart.csv")
df.head()

<>:5: SyntaxWarning: invalid escape sequence '\h'
<>:5: SyntaxWarning: invalid escape sequence '\h'
C:\Users\Chris\AppData\Local\Temp\ipykernel_13304\2833298834.py:5: SyntaxWarning: invalid escape sequence '\h'
  df = pd.read_csv("csvs\heart.csv")


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


## Subtask 1


In [31]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler
from sklearn.model_selection import train_test_split 
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

scalers = [None, StandardScaler(), MinMaxScaler(), MaxAbsScaler()]
accuracies = {}

X = df.drop(columns=["target"])
y = df["target"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

clf = LinearSVC()

for scaler in scalers:
    X_train_scaled = scaler.fit_transform(X_train) if scaler is not None else X_train
    X_test_scaled = scaler.fit_transform(X_test) if scaler is not None else X_test

    clf.fit(X_train_scaled, y_train)
    pred = clf.predict(X_test_scaled)

    col = scaler.__class__.__name__ if scaler is not None else "Initial"
    accuracies[col] = [accuracy_score(y_test, pred)]
accuracies

{'Initial': [0.8021978021978022],
 'StandardScaler': [0.8461538461538461],
 'MinMaxScaler': [0.8021978021978022],
 'MaxAbsScaler': [0.7912087912087912]}

If we scale X_test the accuracies are much bigger, but if it's left alone, the accuracies show down for the scaled models

In [33]:
results = pd.DataFrame(data = accuracies)
results

,Initial,StandardScaler,MinMaxScaler,MaxAbsScaler
0,0.802198,0.846154,0.802198,0.791209


We can slearly see that Standard Sclaer has the best results, while the other sclaers and the initial non scaled dataset are close in performance